# Libraries

In [ ]:
import pandas as pd
import numpy as np
import json
import time 
from IPython.display import display, clear_output
# https://github.com/worldbank/REaLTabFormer
from realtabformer import REaLTabFormer

# GPU

In [ ]:
!nvidia-smi

Thu Oct 26 09:45:40 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.113.01             Driver Version: 535.113.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   28C    P0              N/A /  60W |      6MiB /  8188MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Loading Data

In [ ]:
# Open config file
with open('../config.json', 'r') as f:
    config = json.load(f)

# Load data
df_train = pd.read_csv("/home/tiagociic/Projectos/inovmineral/data/data_francisco/EdgeIIot_train.csv", low_memory=False)

# df_train = pd.read_csv("/home/tiagociic/Projectos/inovmineral/data/data_francisco/DNN-EdgeIIoT-dataset_SMALL.csv")

# Characteristics of the Dataset

In [ ]:
# show head of dataset
df_train.head(10).style.set_properties(**{'text-align': 'left'})

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.request.method,http.referer,http.request.version,http.response,http.tls_port,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.connection.fin,tcp.connection.rst,tcp.connection.syn,tcp.connection.synack,tcp.flags,tcp.flags.ack,tcp.len,tcp.seq,udp.stream,udp.time_delta,dns.qry.name,dns.qry.name.len,dns.qry.qu,dns.qry.type,dns.retransmission,dns.retransmit_request,dns.retransmit_request_in,mqtt.conack.flags,mqtt.conflag.cleansess,mqtt.conflags,mqtt.hdrflags,mqtt.len,mqtt.msg_decoded_as,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_label,Attack_type
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,6.000000,2774423095.000000,313.000000,0.000000,0.000000,0.000000,0.000000,16.000000,1.000000,0.000000,59.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,Normal
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1622034.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1,DDoS_UDP
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,91101.000000,2371715782.000000,60523.000000,0.000000,0.000000,0.000000,0.000000,16.000000,1.000000,1440.000000,22641331.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,Normal
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,3531.000000,2371628212.000000,21486.000000,0.000000,0.000000,0.000000,0.000000,16.000000,1.000000,0.000000,323260.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,Normal
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,5.000000,3323730996.000000,50978.000000,0.000000,0.000000,0.000000,0.000000,24.000000,1.000000,41.000000,15.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,48.000000,39.000000,0.000000,3.000000,0.000000,0,Temperature_and_Humidity,24.000000,0.000000,0.000000,0.000000,0.000000,0,Normal
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.0,0,0.000000,0.000000,303.000000,3453508832.000000,24759.000000,0.000000,0.000000,0.000000,0.000000,16.000000,1.000000,0.000000,262.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1,Password
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,59.000000,466369018.000000,6384.000000,1.000000,0.000000,0.000000,0.000000,17.000000,1.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,Normal
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,1.000000,404378316.000000,37438.000000,0.000000,0.000000,0.000000,0.000000,16.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,Normal
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0

In [ ]:
print(f"Number of columns: {len(df_train.columns)}")
print(f"Number of rows: {len(df_train)}")
# list columns in way intended to be seen in notebook
print(f"Columns: {df_train.columns.values}")

Number of columns: 48
Number of rows: 1527737
Columns: ['arp.opcode' 'arp.hw.size' 'icmp.checksum' 'icmp.seq_le' 'icmp.unused'
 'http.content_length' 'http.request.method' 'http.referer'
 'http.request.version' 'http.response' 'http.tls_port' 'tcp.ack'
 'tcp.ack_raw' 'tcp.checksum' 'tcp.connection.fin' 'tcp.connection.rst'
 'tcp.connection.syn' 'tcp.connection.synack' 'tcp.flags' 'tcp.flags.ack'
 'tcp.len' 'tcp.seq' 'udp.stream' 'udp.time_delta' 'dns.qry.name'
 'dns.qry.name.len' 'dns.qry.qu' 'dns.qry.type' 'dns.retransmission'
 'dns.retransmit_request' 'dns.retransmit_request_in' 'mqtt.conack.flags'
 'mqtt.conflag.cleansess' 'mqtt.conflags' 'mqtt.hdrflags' 'mqtt.len'
 'mqtt.msg_decoded_as' 'mqtt.msgtype' 'mqtt.proto_len' 'mqtt.protoname'
 'mqtt.topic' 'mqtt.topic_len' 'mqtt.ver' 'mbtcp.len' 'mbtcp.trans_id'
 'mbtcp.unit_id' 'Attack_label' 'Attack_type']


In [ ]:
# Number of lines for Attack_type
counts = df_train["Attack_type"].value_counts()
print(counts)
print(f"\nNumber of normal: {counts['Normal']}")
print(f"Number of attacks: {counts.sum() - counts['Normal']}")
print(f"\nSize of df_train: {df_train.memory_usage(deep=True).sum() / 1024**2:.2f} MBytes")

Attack_type
Normal                   1091222
DDoS_UDP                   96966
DDoS_ICMP                  54438
SQL_injection              40755
Vulnerability_scanner      40086
DDoS_TCP                   40053
Password                   39825
DDoS_HTTP                  38916
Uploading                  29534
Backdoor                   19244
Port_Scanning              15915
XSS                        12031
Ransomware                  7763
Fingerprinting               707
MITM                         282
Name: count, dtype: int64

Number of normal: 1091222
Number of attacks: 436515

Size of df_train: 1164.83 MBytes


In [ ]:
# Randomly sample 100000 rows of "Normal" Attack_type
df_normal = df_train[df_train["Attack_type"] == "Normal"].sample(n=100000, random_state=config['random_state'])
# delete rows all rows of "Normal" Attack_type
df_train = df_train[df_train["Attack_type"] != "Normal"]
# add 100000 rows of "Normal" Attack_type to df_train
df_train = pd.concat([df_train, df_normal])

In [ ]:
counts = df_train["Attack_type"].value_counts()
print(counts)
print(f"\nNumber of normal: {counts['Normal']}")
print(f"Number of attacks: {counts.sum() - counts['Normal']}")
print(f"\nSize of df_train: {df_train.memory_usage(deep=True).sum() / 1024**2:.2f} MBytes")

Attack_type
Normal                   100000
DDoS_UDP                  96966
DDoS_ICMP                 54438
SQL_injection             40755
Vulnerability_scanner     40086
DDoS_TCP                  40053
Password                  39825
DDoS_HTTP                 38916
Uploading                 29534
Backdoor                  19244
Port_Scanning             15915
XSS                       12031
Ransomware                 7763
Fingerprinting              707
MITM                        282
Name: count, dtype: int64

Number of normal: 100000
Number of attacks: 436515

Size of df_train: 415.37 MBytes


In [ ]:
# Counts how many rows of each Attack_type are missing (100.000)
counts_dict = {}
for attack in df_train["Attack_type"].unique():
    counts_dict[attack] = 100000 - df_train[df_train["Attack_type"] == attack].shape[0]
counts_dict

{'DDoS_UDP': 3034,
 'Password': 60175,
 'DDoS_TCP': 59947,
 'Backdoor': 80756,
 'DDoS_ICMP': 45562,
 'Port_Scanning': 84085,
 'Vulnerability_scanner': 59914,
 'SQL_injection': 59245,
 'DDoS_HTTP': 61084,
 'Uploading': 70466,
 'XSS': 87969,
 'Ransomware': 92237,
 'MITM': 99718,
 'Fingerprinting': 99293,
 'Normal': 0}

# Discard Redudant Data

In [ ]:
df_attack = df_train[df_train["Attack_type"] != "Normal"]
# del df_train
print(f"\nSize of df_attack before del Attack_label column: {df_attack.memory_usage(deep=True).sum() / 1024**2:.2f} MBytes")
df_attack = df_attack.drop(columns=["Attack_label"])
print(f"Size of df_attack after del Attack_label column: {df_attack.memory_usage(deep=True).sum() / 1024**2:.2f} MBytes")


Size of df_attack before del Attack_label column: 338.59 MBytes
Size of df_attack after del Attack_label column: 335.25 MBytes


# Data type conversion (to save space)

In [ ]:
# calculate the size of df_attack before
size_before = df_attack.memory_usage(deep=True).sum() / 1024**2

# select only the columns that are not object type
df_numeric = df_attack.select_dtypes(exclude=["object"])

# convert all numeric columns to unsigned integer type
df_numeric = df_numeric.apply(pd.to_numeric, downcast="unsigned")

# print df_numeric data types before and after conversion side by side
print(pd.concat([df_attack.dtypes, df_numeric.dtypes], axis=1,keys=["Before", "After"]))

# drop all numeric columns from df_attack
df_attack = df_attack.drop(columns=df_numeric.columns)

# concatenate df_attack and newly convertes df_numeric columns
df_attack = pd.concat([df_attack, df_numeric], axis=1)

# calculate the size of df_attack after conversion
size_after = df_attack.memory_usage(deep=True).sum() / 1024**2

# print size before AND after conversion!
print(f"\nSize of df_attack before data conversion: {size_before:.2f} MBytes, after: {size_after:.2f} MBytes")


/home/tiagociic/Projectos/inovmineral/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:375: RuntimeWarning: invalid value encountered in cast
  new_result = trans(result).astype(dtype)


                            Before    After
arp.opcode                 float64    uint8
arp.hw.size                float64    uint8
icmp.checksum              float64   uint16
icmp.seq_le                float64   uint16
icmp.unused                float64    uint8
http.content_length        float64   uint32
http.request.method         object      NaN
http.referer                object      NaN
http.request.version        object      NaN
http.response              float64    uint8
http.tls_port              float64    uint8
tcp.ack                    float64   uint32
tcp.ack_raw                float64   uint32
tcp.checksum               float64   uint16
tcp.connection.fin         float64    uint8
tcp.connection.rst         float64    uint8
tcp.connection.syn         float64    uint8
tcp.connection.synack      float64    uint8
tcp.flags                  float64    uint8
tcp.flags.ack              float64    uint8
tcp.len                    float64   uint16
tcp.seq                    float

In [ ]:
# print head of df_attack
df_attack.head(10).style.set_properties(**{'text-align': 'left'})

,http.request.method,http.referer,http.request.version,dns.qry.name.len,mqtt.conack.flags,mqtt.protoname,mqtt.topic,Attack_type,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.response,http.tls_port,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.connection.fin,tcp.connection.rst,tcp.connection.syn,tcp.connection.synack,tcp.flags,tcp.flags.ack,tcp.len,tcp.seq,udp.stream,udp.time_delta,dns.qry.name,dns.qry.qu,dns.qry.type,dns.retransmission,dns.retransmit_request,dns.retransmit_request_in,mqtt.conflag.cleansess,mqtt.conflags,mqtt.hdrflags,mqtt.len,mqtt.msg_decoded_as,mqtt.msgtype,mqtt.proto_len,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_UDP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,1622034,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0.0,0,0.0,0.0,0.0,0.0,Password,0,0,0,0,0,0,0,0,303,3453508832,24759,0,0,0,0,16,1,0,262.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_UDP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,2594368,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_UDP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,1871277,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,0,0.0,0,0.0,0.0,0.0,0.0,Password,0,0,0,0,0,0,0,0,1,1080021225,15803,0,0,0,0,16,1,0,1.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_TCP,0,0,0,0,0,0,0,0,2140609672,2140609672,20485,0,0,1,0,2,0,120,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Backdoor,0,0,0,0,0,0,0,0,1,1453306566,6843,0,0,0,0,16,1,1448,1463329.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29,0,0.0,0,0.0,0.0,0.0,0.0,Password,0,0,0,0,0,0,0,0,0,0,16528,0,0,1,0,2,0,0,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_UDP,0,0,0,57101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,10834,0,655220,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS_UDP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,1510333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# separate df_attack by Attack_type into 2 dataframes with similar number of lines
df_attack_1 = df_attack[df_attack["Attack_type"].isin(["Backdoor", 
                                                       "DDoS_HTTP", 
                                                       "DDoS_ICMP", 
                                                       "DDoS_TCP", 
                                                       "DDoS_UDP", 
                                                       "Password"])]

df_attack_2 = df_attack[df_attack["Attack_type"].isin(["Port_Scanning", 
                                                       "SQL_injection", 
                                                       "Uploading", 
                                                       "Vulnerability_scanner", 
                                                       "Ransomware",
                                                       "XSS", 
                                                       "Fingerprinting", 
                                                       "MITM"])]
print("-----------------------")
print("df_attack_1")
# print number of lines per Attack_type for each dataframe and total number of lines for each dataframe
display(df_attack_1['Attack_type'].value_counts().to_frame().style.set_properties(**{'text-align': 'left'}))
# print sum of lines for df_attack_1
print(f"Number of lines: {df_attack_1['Attack_type'].value_counts().sum()}")
print(f"\nSize: {df_attack_1.memory_usage(deep=True).sum() / 1024**2:.2f} MBytes")
print("-----------------------\n")
print("df_attack_2")
# print number of lines per Attack_type for each dataframe and total number of lines for each dataframe
display(df_attack_2['Attack_type'].value_counts().to_frame().style.set_properties(**{'text-align': 'left'}))
# print sum of lines for df_attack_2
print(f"Number of lines: {df_attack_2['Attack_type'].value_counts().sum()}")
print(f"Size: {df_attack_2.memory_usage(deep=True).sum() / 1024**2:.2f} MBytes")
print("-----------------------")
      


-----------------------
df_attack_1


,count
Attack_type,
DDoS_UDP,96966
DDoS_ICMP,54438
DDoS_TCP,40053
Password,39825
DDoS_HTTP,38916
Backdoor,19244


Number of lines: 289442

Size: 154.04 MBytes
-----------------------

df_attack_2


,count
Attack_type,
SQL_injection,40755
Vulnerability_scanner,40086
Uploading,29534
Port_Scanning,15915
XSS,12031
Ransomware,7763
Fingerprinting,707
MITM,282


Number of lines: 147073
Size: 78.81 MBytes
-----------------------


# Training df_attack_1 classes model

In [ ]:
# REalTabFormer instation: non-relational data variant
rtf_model_df1 = REaLTabFormer(model_type="tabular",
                          batch_size=1,
                          gradient_accumulation_steps=1,
                          checkpoints_dir = "rtf_checkpoint_df_attack_1",
                          logging_steps=100,
                          random_state= config['random_state'],
                          epochs= 10,
                          train_size = 0.8,                    # 80% of the data for training, 20% for validation
                          numeric_max_len= 12
                          )

# Note: number of bootstrap samples has influnce in required RAM memory, before optimization

In [ ]:
# train df_attack_1 model
rtf_model_df1.fit(df_attack_1)
# save model
rtf_model_df1.save("rtf_models/")

# Training df_attack_2 classes model

In [ ]:
rtf_model_df2 = REaLTabFormer(model_type="tabular",
                          batch_size=1,
                          gradient_accumulation_steps=1,
                          checkpoints_dir = "rtf_checkpoint_df_attack_2_v2",
                          logging_steps=100,
                          random_state= config['random_state'],
                          epochs= 10,
                          train_size = 0.8,                    # 80% of the data for training, 20% for validation
                          numeric_max_len= 12
                          )

In [ ]:
# train df_attack_2 model
rtf_model_df2.fit(df_attack_2)
# save model
rtf_model_df2.save("rtf_models/")

Computing the sensitivity threshold...
Using parallel computation!!!


/home/tiagociic/Projectos/inovmineral/lib/python3.11/site-packages/realtabformer/realtabformer.py:534: UserWarning: The frac (0.165) set results to a sample larger than                     frac_max_data=10000. Setting frac to 0.06799344543186037.
  warnings.warn(
/home/tiagociic/Projectos/inovmineral/lib/python3.11/site-packages/realtabformer/realtabformer.py:568: UserWarning: Duplicate rate (0.0) in the data is zero. The `qt_interval` will be set                     to qt_interval_unique=100.
  warnings.warn(


Bootstrap round:   0%|          | 0/500 [00:00<?, ?it/s]

Sensitivity threshold summary:
count    500.000000
mean       0.000020
std        0.001220
min       -0.003965
25%       -0.000775
50%       -0.000038
75%        0.000776
max        0.003186
dtype: float64
Sensitivity threshold: 0.0020209899999999994 qt_max: 0.05


Map:   0%|          | 0/147073 [00:00<?, ? examples/s]

...

{'eval_loss': 0.47221875190734863, 'eval_runtime': 90.6136, 'eval_samples_per_second': 324.62, 'eval_steps_per_second': 324.62, 'epoch': 0.04}
{'loss': 0.4811, 'learning_rate': 4.977902055108875e-05, 'epoch': 0.04}


  0%|          | 0/29415 [00:00<?, ?it/s]

{'eval_loss': 0.47343870997428894, 'eval_runtime': 90.5622, 'eval_samples_per_second': 324.804, 'eval_steps_per_second': 324.804, 'epoch': 0.04}
{'train_runtime': 657.5831, 'train_samples_per_second': 1789.249, 'train_steps_per_second': 1789.249, 'train_loss': 0.06301491590646598, 'epoch': 0.04}


  0%|          | 0/30000 [00:00<?, ?it/s]

Generated 0 invalid samples out of total 30080 samples generated. Sampling efficiency is: 100.0000%
Saving not-best model...
Critic round: 10,                     sensitivity_threshold: 0.0020209899999999994,                         val_sensitivity: 0.00399436,                             val_sensitivities: [0.0036542, 0.0032517999999999996, 0.0028190000000000003, 0.0022500000000000003, 0.003990800000000001, 0.005039199999999999, 0.005099600000000001, 0.003762600000000001, 0.0034178000000000004, 0.0046236, 0.0044816, 0.0031036, 0.0047409999999999996, 0.005065000000000001, 0.0046156]
Copying artefacts from: best-disc-model
Copying artefacts from: mean-best-disc-model
Copying artefacts from: not-best-disc-model
Copying artefacts from: last-epoch-model


# Data Generation

In [ ]:
# load best df_attack_1 model (id000016979230192943532032)
rtf_model_df1 = REaLTabFormer.load_from_dir(path="rtf_models/id000016979230192943532032")

In [ ]:
def sample_by_category(model, category_dict, columns):
    # create empty dataframe with columns 
    sampled_df = pd.DataFrame(columns=columns)
    for category, n_samples in category_dict.items():
        start_time = time.time()
        while len(sampled_df[sampled_df['Attack_type'] == category]) < n_samples:
            
            samples = model.sample(n_samples=20000)
            # select only the samples with the desired category
            category_samples = samples[samples['Attack_type'] == category]
            # discard the excess samples if necessary
            if len(category_samples) > n_samples:
                category_samples = category_samples.sample(n=n_samples)
            # add the selected samples to the sampled_df dataframe
            sampled_df = pd.concat([sampled_df, category_samples], ignore_index=True)
            
            elapsed_time = time.time() - start_time
            if elapsed_time > 5:
                clear_output(wait=True)
                print(f"{len(sampled_df[sampled_df['Attack_type'] == category])}/{n_samples}", end="\r")
                start_time = time.time()
    return sampled_df

In [ ]:
counts_df1= {'DDoS_UDP': 3034,
             'DDoS_HTTP':61084,
             'Password': 60175,
             'DDoS_TCP': 59947,
             'Backdoor': 80756,
             'DDoS_ICMP': 45562}

In [ ]:
RTB_dataset_df1 = sample_by_category(rtf_model_df1, counts_df1, df_train.columns)

  0%|          | 0/10000 [00:00<?, ?it/s]

Generated 0 invalid samples out of total 10112 samples generated. Sampling efficiency is: 100.0000%


In [ ]:
# save RTB_dataset_df1 to csv file
RTB_dataset_df1.to_csv("RTB_dataset_df1.csv", index=False)

In [ ]:
counts_df2= {"Port_Scanning": 84085, 
             "SQL_injection":59245 , 
             "Uploading":70466, 
             "Vulnerability_scanner":59914, 
             "Ransomware":92237, 
             "Fingerprinting":99293,
             "XSS": 87969,
             "MITM":99718}



In [ ]:
counts_df2= {"XSS": 87969}
rtf_model_df2 = REaLTabFormer.load_from_dir(path="rtf_models/id000016982514417271404544")
                                            
                                            

In [ ]:
# load best df_attack_1 model (id000016979230192943532032)
rtf_model_df2 = REaLTabFormer.load_from_dir(path="rtf_models/id000016980601259525519360")

In [ ]:
RTB_dataset_df2 = sample_by_category(rtf_model_df2, 
                                     counts_df2, 
                                     df_train.columns)

In [ ]:
# save RTB_dataset_df2 to csv file
RTB_dataset_df2.to_csv("RTB_dataset_df2_XSS.csv", index=False)

# Reconstruct, Concatenate and save augmented Dataset

In [ ]:
# load RTB_dataset_df1 and RTB_dataset_df2
RTB_dataset_df1 = pd.read_csv("RTB_dataset_df1.csv", low_memory=False)
RTB_dataset_df2 = pd.read_csv("RTB_dataset_df2.csv", low_memory=False)
RTB_dataset_df3 = pd.read_csv("RTB_dataset_df2_XSS.csv", low_memory=False)

In [ ]:
# print number of lines per Attack_type for each dataframe and total number of lines for each dataframe
display(RTB_dataset_df1['Attack_type'].value_counts().to_frame().style.set_properties(**{'text-align': 'left'}))

,count
Attack_type,
Backdoor,81549
DDoS_HTTP,62613
Password,62373
DDoS_TCP,60349
DDoS_ICMP,46182
DDoS_UDP,3258


In [ ]:
RTB_dataset_df2.head(10).style.set_properties(**{'text-align': 'left'})

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.request.method,http.referer,http.request.version,http.response,http.tls_port,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.connection.fin,tcp.connection.rst,tcp.connection.syn,tcp.connection.synack,tcp.flags,tcp.flags.ack,tcp.len,tcp.seq,udp.stream,udp.time_delta,dns.qry.name,dns.qry.name.len,dns.qry.qu,dns.qry.type,dns.retransmission,dns.retransmit_request,dns.retransmit_request_in,mqtt.conack.flags,mqtt.conflag.cleansess,mqtt.conflags,mqtt.hdrflags,mqtt.len,mqtt.msg_decoded_as,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_label,Attack_type
0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,1,907620220,17039,0,1,0,0,20,1,0,1.000000,0,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,Port_Scanning
1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,287595410,287595430,12272,0,0,1,0,2,0,0,0.000000,0,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,Port_Scanning
2,0,0,0,0,0,0,0.0,0.0,0.0,0,0,424414631,424414651,48093,0,0,1,0,2,0,34,0.000000,0,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,Port_Scanning
3,0,0,0,0,0,0,0.0,0.0,0.0,0,0,1,1745749404,52488,0,1,0,0,20,1,0,1.000000,0,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,Port_Scanning
4,0,0,0,0,0,0,0.0,0.0,0.0,0,0,1,2158814217,56859,0,1,0,0,20,1,0,1.000000,0,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,Port_Scanning
5,0,0,0,0,0,0,0.0,0.0,0.0,0,0,1,541688840,57305,0,1,0,0,20,1,0,1.000000,0,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,Port_Scanning
6,0,0,0,0,0,0,0.0,0.0,0.0,0,0,143321801,143321801,14311,0,0,1,0,2,0,0,0.000000,0,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,Port_Scanning
7,0,0,0,0,0,0,0.0,0.0,0.0,0,0,197975652,197975652,59956,0,0,1,0,2,0,0,0.000000,0,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,Port_Scanning
8,0,0,0,0,0,0,0.0,0.0,0.0,0,0,1,948882140,57349,0,1,0,0,20,1,0,1.000000,0,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,Port_Scanning
9,0,0,0,0,0,0,0.0,0.0,0.0,0,0,593181452,593181452,23679,0,0,1,0,2,0,0,0.000000,0,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,Port_Scanning


In [ ]:
# print number of lines per Attack_type for each dataframe and total number of lines for each dataframe
display(RTB_dataset_df2['Attack_type'].value_counts().to_frame().style.set_properties(**{'text-align': 'left'}))

,count
Attack_type,
MITM,99812
Fingerprinting,99745
Ransomware,94039
Port_Scanning,84733
Uploading,70749
SQL_injection,61693
Vulnerability_scanner,61128


In [ ]:
# concatenate RTB_dataset_df1 and RTB_dataset_df2
RTB_dataset = pd.concat([RTB_dataset_df1, RTB_dataset_df2], ignore_index=True)

RTB_dataset.head(10).style.set_properties(**{'text-align': 'left'})

,arp.opcode,arp.hw.size,icmp.checksum,icmp.seq_le,icmp.unused,http.content_length,http.request.method,http.referer,http.request.version,http.response,http.tls_port,tcp.ack,tcp.ack_raw,tcp.checksum,tcp.connection.fin,tcp.connection.rst,tcp.connection.syn,tcp.connection.synack,tcp.flags,tcp.flags.ack,tcp.len,tcp.seq,udp.stream,udp.time_delta,dns.qry.name,dns.qry.name.len,dns.qry.qu,dns.qry.type,dns.retransmission,dns.retransmit_request,dns.retransmit_request_in,mqtt.conack.flags,mqtt.conflag.cleansess,mqtt.conflags,mqtt.hdrflags,mqtt.len,mqtt.msg_decoded_as,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,Attack_label,Attack_type
0,0,0,0,0,0,0,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,2322685,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,DDoS_UDP
1,0,0,0,0,0,0,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,2247966,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,DDoS_UDP
2,0,0,0,0,0,0,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,1050240,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,DDoS_UDP
3,0,0,0,0,0,0,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,2902642,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,DDoS_UDP
4,0,0,0,0,0,0,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,12630,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,DDoS_UDP
5,0,0,0,0,0,0,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,140647,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,DDoS_UDP
6,0,0,0,0,0,0,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,2383582,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,DDoS_UDP
7,0,0,0,0,0,0,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,1547642,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,DDoS_UDP
8,0,0,0,0,0,0,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,2260916,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,DDoS_UDP
9,0,0,0,0,0,0,0.0,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,2543942,0,0,0.000000,0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0.000000,0,0,0,0,0,nan,DDoS_UDP


In [ ]:
# print number of lines per Attack_type for each dataframe and total number of lines for each dataframe
display(RTB_dataset['Attack_type'].value_counts().to_frame().style.set_properties(**{'text-align': 'left'}))

,count
Attack_type,
MITM,99812
Fingerprinting,99745
Ransomware,94039
Port_Scanning,84733
Backdoor,81549
Uploading,70749
DDoS_HTTP,62613
Password,62373
SQL_injection,61693


In [ ]:
# save RTB_dataset to csv file
RTB_dataset.to_csv("RTB_dataset.csv", index=False)

# Concatenate df_train and RTB_dataset
df_train = pd.concat([df_train, RTB_dataset], ignore_index=True)

In [ ]:
# print number of lines per Attack_type for each dataframe and total number of lines for each dataframe
display(df_train['Attack_type'].value_counts().to_frame().style.set_properties(**{'text-align': 'left'}))

,count
Attack_type,
SQL_injection,102448
Password,102198
Ransomware,101802
DDoS_HTTP,101529
Vulnerability_scanner,101214
Backdoor,100793
Port_Scanning,100648
DDoS_ICMP,100620
Fingerprinting,100452
